In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import math

opts = webdriver.ChromeOptions()
opts.add_experimental_option('prefs', {
    'download.default_directory': r"C:\Users\USER\Downloads"
})
driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe", options=opts)

In [ ]:
# url = "https://movie.daum.net/moviedb/grade?movieId=83642"
# url = "https://movie.daum.net/moviedb/grade?movieId=93252"
url = "https://movie.daum.net/moviedb/grade?movieId=3212"

driver.get(url)

In [ ]:
# 클릭 수 = (총 평점 수 - 10) / 30(클릭 당 새로 호출되는 평점 수)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

comment_num_txt = soup.find('span', {'class': 'txt_netizen'})
comment_num = int(comment_num_txt.text[1:-2])

click_num = (comment_num - 10) / 30
click_num

In [ ]:
for i in range(math.ceil(click_num)):
    more_button = '//*[@class="link_fold #more"]'
    driver.find_element('xpath', more_button).click()
    time.sleep(2)

In [ ]:
# 댓글 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

tmp_list = []
review = soup.find_all('p', {'class': 'desc_txt font_size_'})

for p in review:
    tmp_list.append(p.text.strip())
    
tmp_list

In [ ]:
len(tmp_list)

In [ ]:
# 별점 가져오기(댓글이 있다면)
comments = soup.find("ul", {"class": "list_comment"}).find_all("li")
star_list = []

for abc in comments:
    if abc.find('p', {'class': 'desc_txt font_size_'}):
        ans = abc.find_all("div")[1]
        star_list.append(ans.text.strip())
        
star_list

In [ ]:
len(star_list)

In [ ]:
# 긍정/부정 댓글 인덱스 저장
pos_index = []
neg_index = []

for i in range(len(star_list)):
    if int(star_list[i]) < 9:
        neg_index.append(i)
    else:
        pos_index.append(i)

In [ ]:
# seperation
pos_list = []
neg_list = []

for i in range(len(pos_index)):
    pos_list.append(tmp_list[pos_index[i]])
    
for i in range(len(neg_index)):
    neg_list.append(tmp_list[neg_index[i]])

In [ ]:
# 분석
from konlpy.tag import Okt  # pip install konlpy
import time


start = time.time()

okt = Okt()

ko_doc = '\n'.join(tmp_list)
ko_doc_noun = okt.nouns(ko_doc)
print(time.time() - start)

In [ ]:
start = time.time()

okt = Okt()

pos_doc = '\n'.join(pos_list)
# print(ko_doc[:1000])
# print(len(ko_doc))

pos_doc_noun = okt.nouns(pos_doc)
print(time.time() - start)

In [ ]:
start = time.time()

okt = Okt()

neg_doc = '\n'.join(neg_list)
# print(ko_doc[:1000])
# print(len(ko_doc))

neg_doc_noun = okt.nouns(neg_doc)
print(time.time() - start)

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image 
import numpy as np 

# pip install wordcloud
count_noun = Counter(ko_doc_noun)

stopword_list = ['영화']

for word in count_noun:
    if len(word) == 1:
        stopword_list.append(word)
        
stopword_list[-5:]

for stopword in stopword_list:
    if stopword in count_noun:
        count_noun.pop(stopword)





wc_noun = WordCloud(background_color='white', max_words=2000, font_path=r'C:\Windows\Fonts\H2MJRE.ttf')
wc_noun = wc_noun.generate_from_frequencies(count_noun)

plt.imshow(wc_noun, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# pip install wordcloud
count_noun = Counter(neg_doc_noun)

stopword_list = ['단어']    # 제외 단어

for word in count_noun:
    if len(word) == 1:
        stopword_list.append(word)
        
stopword_list[-5:]

for stopword in stopword_list:
    if stopword in count_noun:
        count_noun.pop(stopword)

wc_noun = WordCloud(background_color='white', max_words=2000, font_path=r'C:\Windows\Fonts\H2MJRE.ttf')
wc_noun = wc_noun.generate_from_frequencies(count_noun)

plt.imshow(wc_noun, interpolation="bilinear")
plt.axis("off")
plt.show()